# Preparation of the XML for FSLview/Mango atlas

_Leonardo dec 2019_

To be run locally with a python 3 kernel. Instructions [here](https://research.google.com/colaboratory/local-runtimes.html)

In order to use an atlas in FSLview, an XML must be generated,containing 4 values: the ID of the ROI - starting from 0 - and the medoid of the ROI, that is the voxel in the ROI which minimizes the distance across all other voxels in the ROI.

For FSL, coordinates should be provided in the volume space, not in MNI

Once created, the Atlas XML and the folder with the NIFTI image should be placed in `$FSLDIR/data/atlases`

## Import libraries and define the Atlas nifti

In [1]:
import numpy as np
import nibabel as nib
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd
import sys


hemi = 'LH'
atlas_root = 'Schaefer100_ribbon_subcort_' + hemi

bd = '/Users/luser/GoogleDrive/data/atlases/Schaefer100_ribbon_subcort_tracto'
FSL_atlas_dir = bd + '/FSL_version'

resolution = '2mm'
FSL_atlas_image_path = '/' + atlas_root + '/' + atlas_root + '-' + resolution

atlas = nib.load(bd + '/' + atlas_root + '.nii.gz')

Yeo_flavour = '17'
atlas_csv = bd + '/Schaefer100_ribbon_subcort_labels_Yeo' + Yeo_flavour + '_' + hemi + '.csv'
atlas_info = pd.read_csv(atlas_csv)

print(FSL_atlas_image_path)



/Schaefer100_ribbon_subcort_LH/Schaefer100_ribbon_subcort_LH-2mm


## Calculate medoids for each ROI in the atlas

In [2]:
def calculate_medoid(atlas, ROI_numba, space='volume'):
    """
    Usage: calculate_medoid(atlas, ROI_numba, space='volume')

    atlas : a NIFTI image imported with nibabel
    ROI_numba : 1..N ROI ID/label/value in the atlas
    space : 'volume' or 'MNI'
    
    Leonardo dec 2019
    """
    ROI_coords = np.where(atlas.get_data() == ROI_numba)
    ROI_ijks = np.array([ROI_coords[0], ROI_coords[1], ROI_coords[2]]).T

    D = squareform(pdist(ROI_ijks))
    Dsum = np.sum(D, axis=0)
    
    ijk_min = np.argmin(Dsum)
    ijk_medoid = ROI_ijks[ijk_min,:] # volume space
    MNI_coord_medoid = nib.affines.apply_affine(atlas.affine, ijk_medoid) # MNI space
    
    if space == 'MNI':
        medoid = np.concatenate(([ROI_numba], MNI_coord_medoid), axis=0)
    else:
        medoid = np.concatenate(([ROI_numba], ijk_medoid), axis=0)

    return medoid


NROIs = len(np.unique(atlas.get_data())) - 1 # ROI numba start at 1
medoids = np.zeros((NROIs, 4), dtype='int')

for i in np.arange(NROIs):
    ROI_numba = i + 1
    medoids[i,:] = calculate_medoid(atlas, ROI_numba, space='volume')
    
medoids[0:5,]



array([[ 1, 57, 25, 30],
       [ 2, 58, 15, 35],
       [ 3, 48, 18, 35],
       [ 4, 59, 20, 45],
       [ 5, 54, 33, 33]])

## Prepare XML for FSL
Remember to remove 1 from the ROI ID

A small tut on how to write XML in Python is [here](https://pymotw.com/2/xml/etree/ElementTree/create.html)

In [0]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.etree import ElementTree
from xml.dom import minidom


atlas_labels = atlas_info['original_label'].values


def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem)
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

ROOT = Element('atlas')

header = SubElement(ROOT, 'header')

name = SubElement(header, 'name')
name.text = atlas_root

shortname = SubElement(header, 'shortname')
shortname.text = 'S100_subcort_' + hemi

tipo = SubElement(header, 'type')
tipo.text = 'Deterministic'

images = SubElement(header, 'images')

imagefile = SubElement(images, 'imagefile')
imagefile.text = FSL_atlas_image_path 

summaryimagefile = SubElement(images, 'summaryimagefile')
summaryimagefile.text = FSL_atlas_image_path 


data = SubElement(ROOT, 'data')

for i in np.arange(len(atlas_labels)):
  long_label = atlas_labels[i]
  ROI_FSLnumba = str(medoids[i,0] - 1) # FSL numbering starts from 0
  x = str(medoids[i,1])
  y = str(medoids[i,2])
  z = str(medoids[i,3])

  label = SubElement(data, 'label')
  label.text = long_label
  label.set('index', ROI_FSLnumba)
  label.set('x', x)
  label.set('y', y)
  label.set('z', z)


# print(prettify(ROOT))

# tree = ElementTree.ElementTree(ROOT)

xmlstr = minidom.parseString(ElementTree.tostring(ROOT)).toprettyxml(indent="   ")
with open(atlas_root + '.xml', "wb") as f:
    f.write(xmlstr.encode('ISO-8859-1'))


